<a href="https://colab.research.google.com/github/imanubhav/KARUNA_GO/blob/master/COVID19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np

In [0]:
corpus=pd.read_csv("/content/drive/My Drive/Karuna/corpus.csv",encoding='utf-8')

In [0]:
df=corpus.copy()

In [0]:
df=df.dropna(inplace=False)

In [0]:
df.reset_index(drop = True, inplace = True)

In [0]:
!pip install googletrans

In [0]:
from googletrans import Translator
translator = Translator()
df_en = df.copy()

# translate columns' name using rename function
df_en.rename(columns=lambda x: translator.translate(x).text, inplace=True)

In [0]:
translations = {}
for column in df_en.columns:
    # unique elements of the column
    unique_elements = df_en[column].unique()
    for element in unique_elements:
        # add translation to the dictionary
        if type(element)=='str':
            translations[element] = translator.translate(element).text
# modify all the terms of the data frame by using the previously created dictionary
df_en.replace(translations, inplace = True)

# check translation
df_en.head()

,Unnamed: 0,paper_id,title,abstract,text
0,0,af728043c4fdbde717229134e2cd8bada6760176,Development of a real-time reverse transcripti...,Abstract\n\nBackground: Porcine epidemic diarr...,Background\n\nPorcine epidemic diarrhea (PED) ...
1,4,060f83d466d0f77513f8e3d2b55e149894211b64,Evidence of human coronavirus HKU1 and human b...,Abstract\n\nUndiagnosed cases of respiratory t...,Introduction\n\nAcute respiratory tract infect...
2,7,f11b19a9e1845ff5ab66a42d5409f8caed24faa1,"Sequencing, Annotation and Analysis of the Syr...",Abstract\n\nBackground: The Syrian hamster (go...,Introduction\n\nThe Syrian hamster (golden ham...
3,8,ce19f6664fc109d488b095746ef87195906df39d,Development of Recommendations for the Manufac...,"Abstract\n\npublicly funded repositories, such...",Introduction\n\nThe medical use of stem cells ...
4,9,372b5ab8e95678ecb55a283d2786ff899d044cec,Fish disease dynamics in changing rivers: Salm...,Abstract\n\nThe myxozoan parasite Ceratomyxa s...,Introduction\n\nMyxozoans are a group of aquat...


In [0]:
df_en.reset_index(drop = True, inplace = True)

In [0]:
df_en=df_l.copy()

In [0]:

df_en['abstract_cleaned'] =df_en['abstract'].str.replace('\d+', '')
df_en.reset_index(drop = True, inplace = True)

In [0]:
!pip install sentence-transformers

In [0]:
import warnings

from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

warnings.filterwarnings("ignore")
model = SentenceTransformer('bert-base-nli-mean-tokens')

100%|██████████| 405M/405M [00:31<00:00, 12.8MB/s]


In [0]:
abstract_embeddings = model.encode(df_en['abstract_cleaned'])

In [0]:
np.shape(abstract_embeddings)

(16999, 768)

In [0]:
abstract_embeddings[0]

(768,)

In [0]:
import pickle

In [0]:
with open('abstract_embeddings.pkl','wb') as f:
  pickle.dump(abstract_embeddings, f)

In [0]:
!pip install cdqa


In [0]:
# The End-To-End Closed Domain Question Answering System is used here.
# It is available at: https://pypi.org/project/cdqa/

from cdqa.utils.filters import filter_paragraphs
from cdqa.utils.download import download_model, download_bnpp_data
from cdqa.pipeline.cdqa_sklearn import QAPipeline

download_bnpp_data(dir='./data/bnpp_newsroom_v1.1/')
download_model(model='bert-squad_1.1', dir='./models')

In [0]:
import scipy as sc

USING distilbert-base_embeddings

In [0]:
infile=open('/content/distilbert-base_embeddings.pkl','rb')
distilbert_base_embeddings=pickle.load(infile)

In [0]:
model_db= SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

In [0]:
queries = ['What is optimal quarantine period for coronavirus COVID-19',
           'What is range of incubation period for coronavirus SARS-CoV-2 COVID-19 in humans',
           
         'What is effective quarantine period for coronavirus COVID-19',
           'What is percentage of death cases for coronavirus SARS-CoV-2 COVID-19',
           'What is death rate for coronavirus COVID-19 and air pollution',
           'At which temperature coronavirus COVID-19 can survive',
           'How long coronavirus SARS-CoV-2 can survive on plastic surface',
           'What are risk factors for coronavirus COVID-19',
           'What is origin of coronavirus COVID-19',
           'At which temperature coronavirus cannot survive'] 
query_embeddings = model_db.encode(queries)

In [0]:
def find_relevant_articles1(query=None, top_n_papers=50, min_n_papers=3):
    if query == None:
        query = input('Please enter your query...')
    print('\n\n'+'#'*34+' WORKING ON YOUR QUERY! PLEASE WAIT. '+'#'*34+'\n')   
    distances = sc.spatial.distance.cdist(query_embeddings, distilbert_base_embeddings, "cosine")[0]
    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])
    papers_info = {'query':query, 
                   'id':[], 'title':[], 'abstract':[], 'text':[], 'weight':[], 'index':[]}
    for i in range(top_n_papers):
        tup = results[i]
        #print("\n(Score: %.4f)" % (1-tup[1]),"\n")
        papers_info['id'].append(df_en['paper_id'][tup[0]])
        papers_info['title'].append(df_en['title'][tup[0]])
        papers_info['abstract'].append(df_en['abstract'][tup[0]])
        papers_info['text'].append(df_en['text'][tup[0]])
        papers_info['weight'].append(tup[1])
        papers_info['index'].append(tup[0])
    colms = ['date', 'title', 'category', 'link', 'abstract', 'paragraphs']
    df = pd.DataFrame(columns=colms)
    for i in range(len(papers_info['text'])):
        papers_info['text'][i] = papers_info['text'][i].replace('\n\n', ' ')
        CurrentText = papers_info['text'][i]
        CurrentText = CurrentText.split('. ')
        CurrentList = ["None", papers_info['title'][i], "None", "None", papers_info['abstract'][i], CurrentText]
        CurrentList = np.array(CurrentList)
        CurrentList = CurrentList.reshape(1, CurrentList.shape[0])
        CurrentList = pd.DataFrame(data = CurrentList, columns=colms)
        df = pd.concat([df, CurrentList], ignore_index=True)
    df = filter_paragraphs(df)
    # Loading QAPipeline with CPU version of BERT Reader pretrained on SQuAD 1.1
    cdqa_pipeline = QAPipeline(reader='models/bert_qa.joblib')
    # Fitting the retriever to the list of documents in the dataframe
    cdqa_pipeline.fit_retriever(df=df)
    # Sending a question to the pipeline and getting prediction
    query = papers_info['query']
    prediction = cdqa_pipeline.predict(query=query)
    for i in range(top_n_papers):
        if papers_info['title'][i] == prediction[1]:
            pid = papers_info['id'][i]
    response = {query:{'id':pid,'title':prediction[1],'answer':prediction[0],'summary':prediction[2],
                       'important papers':{'id':papers_info['id'],'title':papers_info['title']}}}
    print('QUERY ASKED: {0}\n'.format(query))
    print('ANSWER {0}\n'.format(prediction[0]))
    print('TITLE: {0}\n'.format(prediction[1]))
    print('EXPLAINED IN PAPER: {0}\n'.format(prediction[2]))
    show_paper = np.min([min_n_papers, top_n_papers])
    print('\nTOP {0} PAPERS FOR YOUR REFERENCE: \n'.format(show_paper))
    for i in range(show_paper):
        print('PAPER #{0}. \nID: {1} \nTITLE: {2}\n n'.format(i+1, papers_info['id'][i], papers_info['title'][i]))
        print("\n(SIMILARITY SCORE: %.4f)" % (1-papers_info['weight'][i]),"\n")
        
    return response, papers_info, prediction, results, df

In [0]:
for query in queries:
    response, papers_info, prediction, result_set, df_temp = find_relevant_articles1(query, top_n_papers=70)



################################## WORKING ON YOUR QUERY! PLEASE WAIT. ##################################

QUERY ASKED: What is optimal quarantine period for coronavirus COVID-19

ANSWER 17-day or 18-day

TITLE: Is a 14-day quarantine period optimal for effectively controlling coronavirus disease 2019 (COVID-19)?

EXPLAINED IN PAPER: If taking a quarantine period of 17-day or 18-day, 94.8% and 96.2%, respectively, of the cases developing symptoms will be successfully quarantined.As the outbreak is fast-moving in the world, based on this analysis we recommended that an extension of adults' quarantine period to 17 or 21 days could be more effective


TOP 3 PAPERS FOR YOUR REFERENCE: 

PAPER #1. 
ID: 1f80f32ab88715153f9690b096115e53f2fd6121 
TITLE: Epidemiological parameters of coronavirus disease 2019: a pooled analysis of publicly reported individual data of 1155 cases from seven countries Summary Background
 n

(SIMILARITY SCORE: 0.7448) 

PAPER #2. 
ID: a819c21262d2a7f6dac68f7f9d984

USING bert-base-nli-mean-tokens

In [0]:

def find_relevant_articles(query=None, top_n_papers=20, min_n_papers=3):
    if query == None:
        query = input('Please enter your query...')
    print('\n\n'+'#'*34+' NEW QUESTION '+'#'*34+'\n')   
    single_embedding=model.encode(query)
    distances = sc.spatial.distance.cdist(single_embedding, abstract_embeddings, "cosine")[0]
    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])
    papers_info = {'query':query, 
                   'id':[], 'title':[], 'abstract':[], 'text':[], 'weight':[], 'index':[]}
    for i in range( top_n_papers):
        tup = results[i]
        #print("\n(Score: %.4f)" % (1-tup[1]),"\n")
        papers_info['id'].append(df_en['paper_id'][tup[0]])
        papers_info['title'].append(df_en['title'][tup[0]])
        papers_info['abstract'].append(df_en['abstract'][tup[0]])
        papers_info['text'].append(df_en['text'][tup[0]])
        papers_info['weight'].append(tup[1])
        papers_info['index'].append(tup[0])
    colms = ['date', 'title', 'category', 'link', 'abstract', 'paragraphs']
    df = pd.DataFrame(columns=colms)
    for i in range(len(papers_info['text'])):
        papers_info['text'][i] = papers_info['text'][i].replace('\n\n', ' ')
        CurrentText = papers_info['text'][i]
        CurrentText = CurrentText.split('. ')
        #CurrentList = ["None", papers_info['title'][i], "None", "None", "None", CurrentText]
        CurrentList = ["None", papers_info['title'][i], "None", "None", papers_info['abstract'][i], CurrentText]
        CurrentList = np.array(CurrentList)
        CurrentList = CurrentList.reshape(1, CurrentList.shape[0])
        CurrentList = pd.DataFrame(data = CurrentList, columns=colms)
        df = pd.concat([df, CurrentList], ignore_index=True)
    df = filter_paragraphs(df)
    
    # Loading QAPipeline with CPU version of BERT Reader pretrained on SQuAD 1.1
    cdqa_pipeline = QAPipeline(reader='models/bert_qa.joblib')
    # Fitting the retriever to the list of documents in the dataframe
    cdqa_pipeline.fit_retriever(df=df)
    # Sending a question to the pipeline and getting prediction
    query = papers_info['query']
    prediction = cdqa_pipeline.predict(query=query)
    for i in range(top_n_papers):
        if papers_info['title'][i] == prediction[1]:
            pid = papers_info['id'][i]
    response = {query:{'id':pid,'title':prediction[1],'answer':prediction[0],'summary':prediction[2],
                       'important papers':{'id':papers_info['id'],'title':papers_info['title']}}}
    print('QUESTION: {0}\n'.format(query))
    print('ANSWER {0}\n'.format(prediction[0]))
    print('TITLE: {0}\n'.format(prediction[1]))
    print('DESCRIBED IN PAPER: {0}\n'.format(prediction[2]))
    show_paper = np.min([min_n_papers, top_n_papers])
    print('\nTOP {0} MOST RELEVANT PAPERS RELATED TO THE QUERY:\n'.format(show_paper))
    for i in range(show_paper):
        print('PAPER #{0}. \nID: {1} \nTITLE: {2}\n'.format(i+1, papers_info['id'][i], papers_info['title'][i]))
    return response, papers_info, prediction, results, df

In [0]:
# List of queries
queries = ['What is range of incubation period for coronavirus SARS-CoV-2 COVID-19 in humans',
           'What is optimal quarantine period for coronavirus COVID-19',
         'What is effective quarantine period for coronavirus COVID-19',
           'What is percentage of death cases for coronavirus SARS-CoV-2 COVID-19',
           'What is death rate for coronavirus COVID-19 and air pollution',
           'At which temperature coronavirus COVID-19 can survive',
           'How long coronavirus SARS-CoV-2 can survive on plastic surface',
           'What are risk factors for coronavirus COVID-19',
           'What is origin of coronavirus COVID-19',
           'At which temperature coronavirus cannot survive'] 
query_embeddings = model.encode(queries)

In [0]:
for query in queries:
    response, papers_info, prediction, result_set, df_temp = find_relevant_articles(query, top_n_papers=100)

In [0]:
query=["What is range of incubation period for coronavirus SARS-CoV-2 COVID-19 in humans"]
single_embeddings=model.encode(query)

In [0]:
distances_temp = sc.spatial.distance.cdist(single_embeddings, abstract_embeddings, "cosine")[0]
results_temp = zip(range(len(distances_temp)), distances_temp)
results_temp = sorted(results_temp, key=lambda x: x[1])

In [0]:
papers_info = {'query':query, 'id':[], 'title':[], 'abstract':[], 'text':[], 'weight':[], 'index':[]}

for i in range(1, 20):
  tup = results_temp[i]
  #print("\n(Score: %.4f)" % (1-tup[1]),"\n")
  papers_info['id'].append(df_en['paper_id'][tup[0]])
  papers_info['title'].append(df_en['title'][tup[0]])
  papers_info['abstract'].append(df_en['abstract'][tup[0]])
  papers_info['text'].append(df_en['text'][tup[0]])
  papers_info['weight'].append(tup[1])
  papers_info['index'].append(tup[0])
colms = ['date', 'title', 'category', 'link', 'abstract', 'paragraphs']
df_check = pd.DataFrame(columns=colms)
for i in range(len(papers_info['text'])):
  papers_info['text'][i] = papers_info['text'][i].replace('\n\n', ' ')
  CurrentText = papers_info['text'][i]
  CurrentText = CurrentText.split('. ')
        #CurrentList = ["None", papers_info['title'][i], "None", "None", "None", CurrentText]
  CurrentList = ["None", papers_info['title'][i], "None", "None", papers_info['abstract'][i], CurrentText]
  CurrentList = np.array(CurrentList)
  CurrentList = CurrentList.reshape(1, CurrentList.shape[0])
  CurrentList = pd.DataFrame(data = CurrentList, columns=colms)
  df_check = pd.concat([df_check, CurrentList], ignore_index=True)
df_check = filter_paragraphs(df_check)

In [0]:
cdqa_pipeline = QAPipeline(reader='models/bert_qa.joblib')
    # Fitting the retriever to the list of documents in the dataframe
cdqa_pipeline.fit_retriever(df=df_check)
    # Sending a question to the pipeline and getting prediction
query = papers_info['query']
prediction = cdqa_pipeline.predict(query="What is range of incubation period for coronavirus SARS-CoV-2 COVID-19 in humans")

In [0]:
response, papers_info, prediction, result_set, df_temp = find_relevant_articles("How long coronavirus SARS-CoV-2 can survive on plastic surface", top_n_papers=100)



################################## NEW QUESTION ##################################

(100, 6)
QUESTION: How long coronavirus SARS-CoV-2 can survive on plastic surface

PAPER #1. 
ID: 24c739b85e7c9082d00e9680cf7a99070c427d55 
TITLE: Resistance of Aerosolized Bacterial Viruses to Four Germicidal Products

PAPER #2. 
ID: 5c025d89fc6cf0a72ee801add30ca474f96a2fee 
TITLE: A COMPENDIUM OF REVIEWS IN BIOCHEMISTRY AND MOLECULAR BIOLOGY PUBLISHED IN THE FIRST HALF OF 1991

PAPER #3. 
ID: ab26f03fd219a3da07426f56fecfcc0a20cb0a4e 
TITLE: Supplemental Material to



In [0]:
df_temp['paragraphs'][0]

["Reverse complement of the genomic viral RNA sequence was used to quantify MA15 anti-genome, and the following annotation was used to quantify each viral mRNA : SARS-MA15_ORF1ab Benchmark study of correlation methods.We evaluated several parametric and non-parametric correlation methods to determine the optimal method for detecting co-expressed coding genes sharing similar functions.Seven unsigned co-expression networks were inferred using Pearson's, Spearman's and In total, 11 different co-expression networks were evaluated against 2 reference networks.The GO-based iso-ontological reference network was obtained by linking genes with at least one shared GO biological process (BP)"]

USING TFIDF EMBEDDING

In [0]:
import nltk

In [0]:
import numpy as np
import pandas as pd
import os
import json
import nltk
from math import log, sqrt
from collections import defaultdict
from copy import deepcopy

In [0]:
corpus=df_en

In [0]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
# This processing algorithm can originaly be found at:
# https://github.com/nilayjain/text-search-engine

inverted_index = defaultdict(list)
num_of_documents = len(corpus)
vects_for_docs = []  # we will need nos of docs number of vectors, each vector is a dictionary
document_freq_vect = {}  # sort of equivalent to initializing the number of unique words to 0

# It updates the vects_for_docs variable with vectors of all the documents.
def iterate_over_all_docs():
    print('Processing corpus...')
    for i in range(num_of_documents):
        if np.mod(i, 1000) == 0:
            print('{0} of {1}'.format(str(i).zfill(len(str(num_of_documents))),num_of_documents))
        doc_text = corpus['title'][i] + ' ' + corpus['abstract'][i] + ' ' + corpus['text'][i]
        token_list = get_tokenized_and_normalized_list(doc_text)
        vect = create_vector(token_list)
        vects_for_docs.append(vect)
    print('{0} of {1}'.format(num_of_documents, num_of_documents))

def create_vector_from_query(l1):
    vect = {}
    for token in l1:
        if token in vect:
            vect[token] += 1.0
        else:
            vect[token] = 1.0
    return vect

def generate_inverted_index():
    count1 = 0
    for vector in vects_for_docs:
        for word1 in vector:
            inverted_index[word1].append(count1)
        count1 += 1

def create_tf_idf_vector():
    vect_length = 0.0
    for vect in vects_for_docs:
        for word1 in vect:
            word_freq = vect[word1]
            temp = calc_tf_idf(word1, word_freq)
            vect[word1] = temp
            vect_length += temp ** 2
        vect_length = sqrt(vect_length)
        for word1 in vect:
            vect[word1] /= vect_length

def get_tf_idf_from_query_vect(query_vector1):
    vect_length = 0.0
    for word1 in query_vector1:
        word_freq = query_vector1[word1]
        if word1 in document_freq_vect:
            query_vector1[word1] = calc_tf_idf(word1, word_freq)
        else:
            query_vector1[word1] = log(1 + word_freq) * log(
                num_of_documents)
        vect_length += query_vector1[word1] ** 2
    vect_length = sqrt(vect_length)
    if vect_length != 0:
        for word1 in query_vector1:
            query_vector1[word1] /= vect_length

def calc_tf_idf(word1, word_freq):
    return log(1 + word_freq) * log(num_of_documents / document_freq_vect[word1])

def get_dot_product(vector1, vector2):
    if len(vector1) > len(vector2):
        temp = vector1
        vector1 = vector2
        vector2 = temp
    keys1 = vector1.keys()
    keys2 = vector2.keys()
    sum = 0
    for i in keys1:
        if i in keys2:
            sum += vector1[i] * vector2[i]
    return sum

def get_tokenized_and_normalized_list(doc_text):
    tokens = nltk.word_tokenize(doc_text)
    ps = nltk.stem.PorterStemmer()
    stemmed = []
    for words in tokens:
        stemmed.append(ps.stem(words))
    return stemmed

def create_vector(l1):
    vect = {}  # this is a dictionary
    global document_freq_vect
    for token in l1:
        if token in vect:
            vect[token] += 1
        else:
            vect[token] = 1
            if token in document_freq_vect:
                document_freq_vect[token] += 1
            else:
                document_freq_vect[token] = 1
    return vect

def get_result_from_query_vect(query_vector1):
    parsed_list = []
    for i in range(num_of_documents - 0):
        dot_prod = get_dot_product(query_vector1, vects_for_docs[i])
        parsed_list.append((i, dot_prod))
        parsed_list = sorted(parsed_list, key=lambda x: x[1])
    return parsed_list

iterate_over_all_docs()
generate_inverted_index()
create_tf_idf_vector()

Processing corpus...
00000 of 16999
01000 of 16999
02000 of 16999
03000 of 16999
04000 of 16999
05000 of 16999
06000 of 16999
07000 of 16999
08000 of 16999
09000 of 16999
10000 of 16999
11000 of 16999
12000 of 16999
13000 of 16999
14000 of 16999
15000 of 16999
16000 of 16999
16999 of 16999


In [0]:
def find_relevant_articles_tfidf(query=None, top_n_papers=50, min_n_papers=3):
    if query == None:
        query = input('Please enter your query...')
    print('\n\n'+'*'*34+' PROCESSING NEW QUERY '+'*'*34+'\n')   
    query_list = get_tokenized_and_normalized_list(query)
    query_vector = create_vector_from_query(query_list)
    get_tf_idf_from_query_vect(query_vector)
    result_set = get_result_from_query_vect(query_vector)
    papers_info = {'query':query, 'query list':query_list, 'query vector':query_vector,
                   'id':[], 'title':[], 'abstract':[], 'text':[], 'weight':[], 'index':[]}
    for i in range(1, top_n_papers+1):
        tup = result_set[-i]
        papers_info['id'].append(corpus['paper_id'][tup[0]])
        papers_info['title'].append(corpus['title'][tup[0]])
        papers_info['abstract'].append(corpus['abstract'][tup[0]])
        papers_info['text'].append(corpus['text'][tup[0]])
        papers_info['weight'].append(tup[1])
        papers_info['index'].append(tup[0])
    colms = ['date', 'title', 'category', 'link', 'abstract', 'paragraphs']
    df = pd.DataFrame(columns=colms)
    for i in range(len(papers_info['text'])):
        papers_info['text'][i] = papers_info['text'][i].replace('\n\n', ' ')
        CurrentText = papers_info['text'][i]
        CurrentText = CurrentText.split('. ')
        #CurrentList = ["None", papers_info['title'][i], "None", "None", "None", CurrentText]
        CurrentList = ["None", papers_info['title'][i], "None", "None", papers_info['abstract'][i], CurrentText]
        CurrentList = np.array(CurrentList)
        CurrentList = CurrentList.reshape(1, CurrentList.shape[0])
        CurrentList = pd.DataFrame(data = CurrentList, columns=colms)
        df = pd.concat([df, CurrentList], ignore_index=True)
    df = filter_paragraphs(df)
    # Loading QAPipeline with CPU version of BERT Reader pretrained on SQuAD 1.1
    cdqa_pipeline = QAPipeline(reader='models/bert_qa.joblib')
    # Fitting the retriever to the list of documents in the dataframe
    cdqa_pipeline.fit_retriever(df=df)
    # Sending a question to the pipeline and getting prediction
    query = papers_info['query']
    prediction = cdqa_pipeline.predict(query=query)
    for i in range(top_n_papers):
        if papers_info['title'][i] == prediction[1]:
            pid = papers_info['id'][i]
    response = {query:{'id':pid,'title':prediction[1],'answer':prediction[0],'summary':prediction[2],
                       'important papers':{'id':papers_info['id'],'title':papers_info['title']}}}
    print('QUERY: {0}\n'.format(query))
    print('ANSWER MINED FROM PAPER: {0}\n'.format(prediction[0]))
    print('PAPER TITLE: {0}\n'.format(prediction[1]))
    print('PARAGRAPH IN PAPER: {0}\n'.format(prediction[2]))
    show_paper = np.min([min_n_papers, top_n_papers])
    print('\nTOP {0} MOST RELEVANT PAPERS RELATED TO THE QUERY:\n'.format(show_paper))
    for i in range(show_paper):
        print('PAPER #{0}. \nID: {1} \nTITLE: {2}\n'.format(i+1, papers_info['id'][i], papers_info['title'][i]))
    return response, papers_info, prediction, result_set, df

In [0]:
queries = ['What is range of incubation period for coronavirus SARS-CoV-2 COVID-19 in humans',
           'What is optimal quarantine period for coronavirus COVID-19',
           'What is effective quarantine period for coronavirus COVID-19',
           'What is percentage of death cases for coronavirus SARS-CoV-2 COVID-19',
           'What is death rate for coronavirus COVID-19 and air pollution',
           'At which temperature coronavirus COVID-19 can survive',
           'How long coronavirus SARS-CoV-2 can survive on plastic surface',
           'What are risk factors for coronavirus COVID-19',
           'What is origin of coronavirus COVID-19',
           'At which temperature coronavirus cannot survive']

In [0]:
for query in queries:
    response, papers_info, prediction, result_set, df = find_relevant_articles_tfidf(query, top_n_papers=50)



********************************** PROCESSING NEW QUERY **********************************

QUERY: What is range of incubation period for coronavirus SARS-CoV-2 COVID-19 in humans

ANSWER MINED FROM PAPER: five days on average before onset of illness

PAPER TITLE: First reported nosocomial outbreak of severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2) in a pediatric dialysis unit

PARAGRAPH IN PAPER: While persistence of SARS-CoV-2 has been described on inert surfaces, person-to-person transmission via droplets is believed to be the main mode of transmission followed by an incubation period of five days on average before onset of illness [2, 7, 8] .Several publications have described nosocomial transmission for the current COVID-19 pandemic [9, 10]


TOP 3 MOST RELEVANT PAPERS RELATED TO THE QUERY:

PAPER #1. 
ID: 5de6b4de529c8d5c4b015286f3f2ff85f521733f 
TITLE: Journal Pre-proof Current development of COVID-19 diagnostics, vaccines and therapeutics Current development of C

In [0]:
#rank-bm25 model

In [0]:
!pip install rank_bm25

In [0]:
from sentence_transformers import models, losses

In [0]:
word_embedding_model = models.Transformer('bert-base-uncased')

# Apply mean pooling to get one fixed sized sentence vector
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=True,
                               pooling_mode_cls_token=False,
                               pooling_mode_max_tokens=False)

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [0]:
model2_embeddings = model.encode(df_en['text'][:5000],show_progress_bar=True)

In [0]:
import pickle
with open('word_pooling_embeddings.pkl','wb') as f:
  pickle.dump(model2_embeddings, f)

In [0]:
# This piece of code was adopted from the original source at:
# https://www.kaggle.com/bkochetov/answer-extractor-
def find_relevant_articles1(query=None, top_n_papers=50, min_n_papers=3):
    if query == None:
        query = input('Please enter your query...')
    print('\n\n'+'#'*34+' PROCESSING YOUR QUERY '+'#'*34+'\n')   
    #query_list = get_tokenized_and_normalized_list(query)
    #query_vector = create_vector_from_query(query_list)
    #get_tf_idf_from_query_vect(query_vector)
    #result_set = get_result_from_query_vect(query_vector)
    distances = sc.spatial.distance.cdist(query_embeddings, model2_embeddings, "cosine")[0]
    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])
    papers_info = {'query':query, 
                   'id':[], 'title':[], 'abstract':[], 'text':[], 'weight':[], 'index':[]}
    for i in range(top_n_papers):
        tup = results[i]
        #print("\n(Score: %.4f)" % (1-tup[1]),"\n")
        papers_info['id'].append(df_en['paper_id'][tup[0]])
        papers_info['title'].append(df_en['title'][tup[0]])
        papers_info['abstract'].append(df_en['abstract'][tup[0]])
        papers_info['text'].append(df_en['text'][tup[0]])
        papers_info['weight'].append(tup[1])
        papers_info['index'].append(tup[0])
    colms = ['date', 'title', 'category', 'link', 'abstract', 'paragraphs']
    df = pd.DataFrame(columns=colms)
    for i in range(len(papers_info['text'])):
        papers_info['text'][i] = papers_info['text'][i].replace('\n\n', ' ')
        CurrentText = papers_info['text'][i]
        CurrentText = CurrentText.split('. ')
        #CurrentList = ["None", papers_info['title'][i], "None", "None", "None", CurrentText]
        CurrentList = ["None", papers_info['title'][i], "None", "None", papers_info['abstract'][i], CurrentText]
        CurrentList = np.array(CurrentList)
        CurrentList = CurrentList.reshape(1, CurrentList.shape[0])
        CurrentList = pd.DataFrame(data = CurrentList, columns=colms)
        df = pd.concat([df, CurrentList], ignore_index=True)
    df = filter_paragraphs(df)
    # Loading QAPipeline with CPU version of BERT Reader pretrained on SQuAD 1.1
    cdqa_pipeline = QAPipeline(reader='models/bert_qa.joblib')
    # Fitting the retriever to the list of documents in the dataframe
    cdqa_pipeline.fit_retriever(df=df)
    # Sending a question to the pipeline and getting prediction
    query = papers_info['query']
    prediction = cdqa_pipeline.predict(query=query)
    for i in range(top_n_papers):
        if papers_info['title'][i] == prediction[1]:
            pid = papers_info['id'][i]
    response = {query:{'id':pid,'title':prediction[1],'answer':prediction[0],'summary':prediction[2],
                       'important papers':{'id':papers_info['id'],'title':papers_info['title']}}}
    print('QUESTION: {0}\n'.format(query))
    print('ANSWER {0}\n'.format(prediction[0]))
    print('TITLE: {0}\n'.format(prediction[1]))
    print('DESCRIBED IN PAPER: {0}\n'.format(prediction[2]))
    show_paper = np.min([min_n_papers, top_n_papers])
    print('\nTOP {0} MOST RELEVANT PAPERS RELATED TO THE QUERY:\n'.format(show_paper))
    for i in range(show_paper):
        print('PAPER #{0}. \nID: {1} \nTITLE: {2}\n n'.format(i+1, papers_info['id'][i], papers_info['title'][i]))
        print("\n(Score: %.4f)" % (1-papers_info['weight'][i]),"\n")
        
    return response, papers_info, prediction, results, df

In [0]:
query_embeddings = model.encode(queries)

In [0]:
for query in queries:
    response, papers_info, prediction, result_set, df_temp = find_relevant_articles1(query, top_n_papers=100)



################################## PROCESSING YOUR QUERY ##################################

QUESTION: What is range of incubation period for coronavirus SARS-CoV-2 COVID-19 in humans

ANSWER less than three months

TITLE: To appear in: Infection Prevention in Practice

DESCRIBED IN PAPER: The virus was identified as a novel enveloped RNA betacoronavirus that has been named as Severe Acute Respiratory Syndrome Coronavirus 2 (SARS-CoV-2) (2) .One of the characteristics of COVID-19 is that it is highly contagious; many countries were affected, including China and 164 other countries in less than three months


TOP 3 MOST RELEVANT PAPERS RELATED TO THE QUERY:

PAPER #1. 
ID: eb5bfeea7bc4bb15dcf2ad3e26dc6478c277fd6e 
TITLE: "Amantadine disrupts lysosomal gene expression; potential therapy for COVID19"
 n

(Score: 0.8863) 

PAPER #2. 
ID: 78ea5fa9437b7e90314416f066a38ad9b2d6e4cf 
TITLE: Genotype and phenotype of COVID-19: Their roles in pathogenesis KEYWORDS
 n

(Score: 0.8770) 

PAPER #3